# Creating a file containing all station meta data
This Jupyter notebook reads two files from NOAA: ghcnd-stations.txt and ghcnd-inventory.txt. The first contains station physical information (latitude, longtidue, elevation), and the second, the climate variables recorded by each station over what time period. 

This notebook merges these two files and creates a single file called gchnd_station_master_ts_max.csv that includes a list of all of the stations which includes data for the TMAX variable. Right now, no non-temperature variables are included, and of the three temperature variables NOAA records (TMAX, TMIN, TAVG), TAVG is based upon TMAX and TMIN and therefore reundant (TAVG = (TMAX + TMIN)/2) and also not as complete, and we need at least TMAX to calculate the "TMID" values I use here. So limiting this file to TMAX preserves all the data we actually need (and no more), making the processing faster when you look to find and load a station. 

In [29]:
#This cell imports the right packages.

import pandas as pd
import yaml
#This YAML file contains a great deal of static information, 
#such as directory information. 
yaml_file = open("filenames.yaml")
yaml = yaml.load(yaml_file, Loader=yaml.FullLoader)


In [30]:
#This loads the file containing all of the station meta information, including lat/lon.

allstations = pd.read_fwf(yaml['GHCND_STATIONS'],colspecs=[
                                                                          #ID#
                                                                        (0,11),
                                                                        #Lat/Lon
                                                                        (11,20),(21,30),
                                                                        #Eelvation, tength of meters
                                                                        (31,37),
                                                                        #Station name
                                                                        (38,72),
                                                                        #Other
                                                                        (73,82),(83,85)
                                                                        ])
#Name the columns Correctly.
allstations.columns=['ID','Latitude','Longitude','Elevation (0.1m)','Name','Other1','Other2']
allstations

,ID,Latitude,Longitude,Elevation (0.1m),Name,Other1,Other2
0,ACW00011647,17.1333,-61.7833,19.2,ST JOHNS,NaN,NaN
1,AE000041196,25.3330,55.5170,34.0,SHARJAH INTER. AIRP,SN 41,96.0
2,AEM00041194,25.2550,55.3640,10.4,DUBAI INTL,41,94.0
3,AEM00041217,24.4330,54.6510,26.8,ABU DHABI INTL,41,17.0
4,AEM00041218,24.2620,55.6090,264.9,AL AIN INTL,41,18.0
...,...,...,...,...,...,...,...
119497,ZI000067969,-21.0500,29.3670,861.0,WEST NICHOLSON,67,69.0
119498,ZI000067975,-20.0670,30.8670,1095.0,MASVINGO,67,75.0
119499,ZI000067977,-21.0170,31.5830,430.0,BUFFALO RANGE,67,77.0
119500,ZI000067983,-20.2000,32.6160,1132.0,CHIPINGE,SN 67,83.0


In [23]:
#This loads the datafile containing the time periods of analysis
stationperiods = pd.read_fwf(yaml['GHCND_INVENTORY'],colspecs=[
                                                                         #Country code#
                                                                        (0,2),                                                                       
                                                                        #ID#
                                                                        (0,11),
                                                                        #Lat/Lon
                                                                        (11,20),(21,30),
                                                                        #Others
                                                                        (31,35),
                                                                        #First year, Last Year
                                                                        (36,40),(41,45)
                                                                        ])
stationperiods.columns=['Country','ID','Latitude','Longitude','Element','Firstyear','Lastyear']
stationperiods

,Country,ID,Latitude,Longitude,Element,Firstyear,Lastyear
0,AC,ACW00011604,17.1167,-61.7833,TMIN,1949,1949
1,AC,ACW00011604,17.1167,-61.7833,PRCP,1949,1949
2,AC,ACW00011604,17.1167,-61.7833,SNOW,1949,1949
3,AC,ACW00011604,17.1167,-61.7833,SNWD,1949,1949
4,AC,ACW00011604,17.1167,-61.7833,PGTM,1949,1949
...,...,...,...,...,...,...,...
709838,ZI,ZI000067983,-20.2000,32.6160,PRCP,1951,2020
709839,ZI,ZI000067983,-20.2000,32.6160,TAVG,1962,2020
709840,ZI,ZI000067991,-22.2170,30.0000,TMAX,1951,1990
709841,ZI,ZI000067991,-22.2170,30.0000,TMIN,1951,1990


In [24]:
#This merges the data into one DataFrame containing all station related data, including all columns, and all elements, smashed together into one enormous
#dataframe (witih over 700,000 rows). 
allstdata = pd.merge(allstations,stationperiods)
allstdata

,ID,Latitude,Longitude,Elevation (0.1m),Name,Other1,Other2,Country,Element,Firstyear,Lastyear
0,ACW00011647,17.1333,-61.7833,19.2,ST JOHNS,NaN,NaN,AC,TMAX,1961,1961
1,ACW00011647,17.1333,-61.7833,19.2,ST JOHNS,NaN,NaN,AC,TMIN,1961,1961
2,ACW00011647,17.1333,-61.7833,19.2,ST JOHNS,NaN,NaN,AC,PRCP,1957,1970
3,ACW00011647,17.1333,-61.7833,19.2,ST JOHNS,NaN,NaN,AC,SNOW,1957,1970
4,ACW00011647,17.1333,-61.7833,19.2,ST JOHNS,NaN,NaN,AC,SNWD,1957,1970
...,...,...,...,...,...,...,...,...,...,...,...
709828,ZI000067983,-20.2000,32.6160,1132.0,CHIPINGE,SN 67,83.0,ZI,PRCP,1951,2020
709829,ZI000067983,-20.2000,32.6160,1132.0,CHIPINGE,SN 67,83.0,ZI,TAVG,1962,2020
709830,ZI000067991,-22.2170,30.0000,457.0,BEITBRIDGE,67,91.0,ZI,TMAX,1951,1990
709831,ZI000067991,-22.2170,30.0000,457.0,BEITBRIDGE,67,91.0,ZI,TMIN,1951,1990


In [25]:
#Then it creates a dataframe that strips variables of interest. 
sttempdata=allstdata[['ID','Latitude','Longitude','Name', 'Element', 'Firstyear', 'Lastyear']]
sttempdatalg=sttempdata[allstdata['Element'].isin(["TAVG","TMAX","TMIN","PRCP","SNOW"])]
sttempdatast=sttempdata[allstdata['Element'].isin(["TMAX","TMIN","PRCP","SNOW"])]
t1=sttempdata[allstdata['Element'].isin(["TMAX","TMIN"])]
onlytempdata=sttempdata[allstdata['Element'].isin(["TMAX","TMIN"])]
onlytmaxdata=sttempdata[allstdata['Element'].isin(["TMAX"])]
#Comparing the dataframe with and without the "TAVG" variable, you can see that there are only 9,000-10,000 or so
#rows with "TAVG". This variable is computed from TMAX and TMIN and only for a small subset of all rows. 
#(There are 80,000 with rows with TMAX or TMIN)
sttempdatalg.info()
sttempdatast.info()
t1.info()
onlytempdata.info()
#use sttempdatast


<class 'pandas.core.frame.DataFrame'>
Int64Index: 278066 entries, 0 to 709832
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   ID         278066 non-null  object 
 1   Latitude   278066 non-null  float64
 2   Longitude  278066 non-null  float64
 3   Name       278066 non-null  object 
 4   Element    278066 non-null  object 
 5   Firstyear  278066 non-null  int64  
 6   Lastyear   278066 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 17.0+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 268611 entries, 0 to 709832
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   ID         268611 non-null  object 
 1   Latitude   268611 non-null  float64
 2   Longitude  268611 non-null  float64
 3   Name       268611 non-null  object 
 4   Element    268611 non-null  object 
 5   Firstyear  268611 non-null  int64  
 6   Lastyear   2686

In [32]:
#Now the dataframe is written to a CSV file. This file includes PRCP and SNOW.
sttempdatast.to_csv(yaml['STATION_MASTER_ALL'])

#Now the dataframe is written to a CSV file. This file includes only TMAX. This file is most useful,
#because it is used to find stations, not read data, so any file with more than one element is redundant
#and takes longer to read.
onlytmaxdata.to_csv(yaml['STATIONMETA'])

In [33]:
#Test the file
test=pd.read_csv(yaml['STATIONMETA'])
test

,Unnamed: 0,ID,Latitude,Longitude,Name,Element,Firstyear,Lastyear
0,0,ACW00011647,17.1333,-61.7833,ST JOHNS,TMAX,1961,1961
1,7,AE000041196,25.3330,55.5170,SHARJAH INTER. AIRP,TMAX,1944,2021
2,11,AEM00041194,25.2550,55.3640,DUBAI INTL,TMAX,1983,2021
3,15,AEM00041217,24.4330,54.6510,ABU DHABI INTL,TMAX,1983,2021
4,19,AEM00041218,24.2620,55.6090,AL AIN INTL,TMAX,1994,2021
...,...,...,...,...,...,...,...,...
40140,709815,ZI000067969,-21.0500,29.3670,WEST NICHOLSON,TMAX,1962,1990
40141,709818,ZI000067975,-20.0670,30.8670,MASVINGO,TMAX,1951,2020
40142,709823,ZI000067977,-21.0170,31.5830,BUFFALO RANGE,TMAX,1968,1990
40143,709826,ZI000067983,-20.2000,32.6160,CHIPINGE,TMAX,1951,2020
